# 3D Segmentation of Brain Tumor

This document is to propose a 2D methodology instead of treating it as a 3D input.


## Loading the Dataset
The dataset used is the BraTS 2020 Dataset.

The Dataset Contains the Following Scans per case:

T1: T1-weighted, native image, sagittal or axial 2D acquisitions, with 1–6 mm slice thickness.

T1c: T1-weighted, contrast-enhanced (Gadolinium) image, with 3D acquisition and 1 mm isotropic voxel size for most patients.

T2: T2-weighted image, axial 2D acquisition, with 2–6 mm slice thickness.

FLAIR: T2-weighted FLAIR image, axial, coronal, or sagittal 2D acquisitions, 2–6 mm slice thickness.

So for training T1ce, T2 and FLAIR will be used.

In [18]:
# Previewing the MRI Data
import numpy as np
import matplotlib.pyplot as plt
from os import path, listdir
from monai.data import Dataset
from monai.transforms import (
    LoadImaged,
    Compose
)

In [8]:
# Prepare list of All training Cases
TRAINING_DATASET_PATH = "./MICCAI_BraTS2020_TrainingData/"
cases = [path.join(TRAINING_DATASET_PATH, x) for x in listdir(TRAINING_DATASET_PATH)]


In [10]:
# Prepare a list of dictionaries of T1CE, T2, FLAIR and Ground Truth Segmentation for preparing the Dataset to be used in the Data Loader 
mri_cases_dictionary = [
    {
        "image": [
            path.join(case, f"{path.split(case)[-1]}_t1ce.nii.gz"),
            path.join(case, f"{path.split(case)[-1]}_t2.nii.gz"),
            path.join(case, f"{path.split(case)[-1]}_flair.nii.gz"),
        ],
        "seg": path.join(case, f"{path.split(case)[-1]}_seg.nii.gz"),
    }
    for case in cases[:30]  #! Only 30 cases taken as test slice
]

# Transform data such as from niftii into Tensors
transform_training_dataset = Compose(
    [
        LoadImaged(keys=["image","seg"])
    ]
)

dataset_training = Dataset(mri_cases_dictionary)

In [17]:
dataset_training[0]['image']

['./MICCAI_BraTS2020_TrainingData/BraTS20_Training_001\\BraTS20_Training_001_t1ce.nii.gz',
 './MICCAI_BraTS2020_TrainingData/BraTS20_Training_001\\BraTS20_Training_001_t2.nii.gz',
 './MICCAI_BraTS2020_TrainingData/BraTS20_Training_001\\BraTS20_Training_001_flair.nii.gz']

In [ ]:
from monai.data import Dataset, CacheDataset,PersistentDataset, decollate_batch, list_data_collate
from glob import glob
from os import path, listdir
import pytorch_lightning as pl
from sklearn.model_selection import ShuffleSplit

from monai.transforms import (
    LoadImaged,
    SpatialPadd,
    EnsureChannelFirstd,
    Compose,
    NormalizeIntensityd,
    Orientationd,
    Spacingd,
    RandSpatialCropd,
    RandFlipd,
    MapTransform,
    CropForegroundd,
    EnsureTyped,
    EnsureType,
    Activations,
    AsDiscrete,
    RandCropByPosNegLabeld,
    RandCropByLabelClassesd,
    RandSpatialCropSamplesd
)

from monai.data import partition_dataset
from monai.networks.nets import UNet,UNETR, SegResNet, BasicUnet
from monai.losses import TverskyLoss, DiceLoss
from monai.metrics import DiceMetric
from monai.inferers import sliding_window_inference
import torch
import matplotlib.pyplot as plt
import numpy as np
import utilities.split_data as split_data
plt.style.use(['science','ieee'])
%matplotlib inline